In [1]:
from hello.hello3 import HelloApp, NotLoggedInError
import datetime
from hello import hello_logger
import os
import requests
import time

In [2]:
# General functions

def call(f, *a, _quiet=False, **k):
    if not _quiet:
        now = datetime.datetime.now().strftime("%H:%M:%S")
        s = "(" + ", ".join(map(repr, a)) + ")"
        print("%s Calling %s%s" % (now, f.__name__, s))
    while True:
        try:
            return f(*a, **k)
        except NotLoggedInError:
            h.login()
        except requests.exceptions.Timeout as e:
            print("Got timeout! Make sure you're on the right network!")
        except (BadError, IOError, requests.exceptions.ConnectionError) as e:
            print("Error occurred:", e.__class__)   
        time.sleep(1)
            
def connect(ip, *args):
    global h
    h = HelloApp(ip, *args)
    
def dfmt(d):
    return d.strftime("%m/%d/%Y %I:%M:%S %p")
    
def wait1(s):
    if s <= 0: return
    dend = dfmt(datetime.datetime.now() + datetime.timedelta(seconds=s))
    print("Sleeping %d seconds, ending at %s" %(s,dend))
    start = time.time()
    end = start + s
    while time.time() < end:
        time.sleep(5)
    
def wait3(s, src):
    start = datetime.datetime.now()
    end = start + datetime.timedelta(seconds=s)
    m1 = dfmt(start)
    m2 = dfmt(end)
    print("\r(%s) Sleeping %d seconds from %s to %s          " % (src, s, m1, m2), end="")
    tend = time.time() + s
    while time.time() < tend:
        time.sleep(5)
    
def wait2(s, *a, **kw):
    pass
wait = wait1

In [3]:
# Specific functions

# globals
logger = None
_mv = None

def setconfig(g, s, v):
    call(h.setconfig, g, s, v)

def getmv():
    global _mv
    _mv = call(h.gpmv, _quiet=True)
    return _mv
    
def setup_controls(rpm=18):
    setag(rpm)
    call(h.setmg, 2, 0)
    
def setag(rpm):
    call(h.setag, 0, rpm)
    
def settemp(t):
    while True:
        call(h.settemp, 0, t)
        mv = getmv()
        tm = mv['temperature']
        if tm['sp'] == t and tm['mode'] == 0:
            break
        else:
            print("Retrying call, failed to set temp")
    
def tempoff():
    call(h.settemp, 2, getmv()['temperature']['sp'])
    
def endbatch():
    if call(h.batchrunning):
        call(h.endbatch)
    
def startbatch(n):
    endbatch()
    call(h.startbatch, n)
    
def temppv():
    return getmv()['temperature']['pv']
    
def waittemp_lower(t, timeout=8*3600):
    waited = 0
    while temppv() > t and (timeout<=0 or waited<timeout):
        wait3(60, 'waittemp_lower')
        waited += 60
    print()
    
def waittemp_higher(t, timeout=8*3600):
    waited = 0
    while temppv() < t and (timeout<=0 or waited<timeout):
        wait3(60, 'waittemp_higher')
        waited += 60
    print()  

    
def rampup(sp, pm=0.05, timeout=4*3600):
    settemp(sp)
    waittemp_higher(sp-pm, timeout)
    
def rampdown(sp, pm=0.0, timeout=10*3600):
    tempoff()
    waittemp_lower(sp+pm, timeout)
    
def newbatch(i, j):
    startbatch(batch_tmpl%(i,j))

In [4]:
def setup_logger(interval=10):
    global logger
    keys = [
        'temperature.pv',
        'condenser.pv',
        'agitation.pv',
#         'MFCs.air',
#         'MFCs.n2',
#         'MFCs.o2',
#         'MFCs.co2',
        'level.pv',
    ]
    if logger is not None:
        logger.stop()
    logger = hello_logger.HelloLogger(h, interval, None, keys)
    logger.start()
    
def logfile(fn):
    print("Creating new file: %r"%fn)
    logger.new_file(fn)
    
def endlog():
    logger.new_file(None)

In [5]:
def ntests(n):
    return range(1,n+1)

### Test Utilities

In [6]:
fn_tmpl = "Temp thermowell test %s %s %s.csv"

def new_file(typ, name):
    logfile(fn_tmpl% (typ, name, bt()))

def dotest(mg, o2, i):
    new_file("rampdown", i)
    rampdown(mg)
    endlog()
    flushheadspace('air')
    new_file("rampup", i)
    rampup(mg, o2)
    endlog()
    dooff()
    
def temptest(name, i):
    new_file(name, i)
    rampup(32)
    wait(2*3600)
    rampup(37)
    wait(2*3600)
    endlog()
    tempoff()
    
def temptest2(name, i):
    new_file(name, i)
    rampup(37)
    wait(2*3600)
    endlog()
    tempoff()
    
def bt(): 
    return datetime.datetime.now().strftime("%y%m%d")

def mkfldr(p):
    path = "C:\\Users\\natha\\Documents\\test\\"+p
    os.makedirs(path, exist_ok=True)
    os.chdir(path)
    
def chd(p):
    os.chdir("C:\\Users\\natha\\Documents\\test\\"+p)

### Pre-Test Setup

In [ ]:
connect('192.168.1.14')
setup_controls(18)
wait(10*3600)
setup_logger()
mkfldr("Plastic Thermowell Test")
temptest2("Ramp Test", 1)    

12:42:09 Calling setag(0, 18)
12:42:12 Calling setmg(2, 0)
Sleeping 36000 seconds, ending at 09/10/2018 10:42:13 PM


In [ ]:
connect('192.168.1.81')
setup_controls(32)
setup_logger()
wait(10*60)
mkfldr("Sam Thermowell Test")
temptest2("Ramp Test RT to 37", 1)    

In [7]:
connect('192.168.1.17')
setup_controls(32)
setup_logger()
#wait(10*60)
mkfldr("Server Test")
temptest2("Ramp Test RT to 37", 1)    

11:59:41 Calling setag(0, 32)
11:59:44 Calling setmg(2, 0)
Creating new file: 'Temp thermowell test Ramp Test RT to 37 1 170831.csv'
11:59:45 Calling settemp(0, 37)
(waittemp_higher) Sleeping 60 seconds from 08/31/2017 02:11:05 PM to 08/31/2017 02:12:05 PM          
Sleeping 7200 seconds, ending at 08/31/2017 04:12:06 PM
16:12:10 Calling settemp(2, 37)


NameError: name 'BadError' is not defined